# MF4056 - Computational Finance - Assignment 1 

### Ben McEnery - 120411954



In [4]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d;
from scipy.stats import multivariate_normal
from ipywidgets import interact
from IPython.display import display, Math

rng=np.random.default_rng()

## Question 1

#### Generating trajectories of r.

solution to the vasicek model is given by:
$$ r(t) = r_{u}e^{-\alpha (t-u)} + b(1-e^{-\alpha(t-u)}) + \sigma\int_{u}^{t} e^{-\alpha(t-u)}dW(u)
$$
Since we have a deterministic function inside the integral $w.r.t$ Brownian motiom $W(t)$
$\implies r(t)$ is gaussian and we can compute it by simulating trajectories from:

$$ r(t_{i+1}) \sim \mathcal{N}\left(r(t_i) e^{-\alpha t_i} + b(1-e^{-\alpha t_i}), \frac{\sigma^2}{2\alpha}(1-e^{-2\alpha t_i})\right)
$$

Will approximate a solution to the Vasicek model as follows:
$$ r_{t_{i+1}} = r_t e^{-\alpha dt} + b(1-e^{-\alpha dt}) + \left(\frac{\sigma^2}{2\alpha}(1-e^{-2\alpha dt})\right)^{0.5} B_{t_i},
$$
where $B_{t_i} \sim \mathcal{N}(0,1)$

In [48]:
alpha= 1
b = 0.03
sigma = 0.05
N = 10
S=10
N=100
M=100
dt=S/N
r = np.zeros((M,N))
t = np.linspace(0, S, N)

In [49]:
def vasicek(r0 = 0.03):
    # intialise the variables
    S = 100
    N = 1000
    dt = S/N
    M = 10
    r = np.zeros((M,N+1))
    r[:,0] = [r0]
    count = 0

    # compute trajectories of r

    W = np.random.normal(0,1,(M, N+1)) # Simulate Brownian motion 
    for i in range(0,N):
        r[:,i+1] = np.exp(-alpha*dt)*(r[:,i]) + b*(1-np.exp(-alpha*dt)) + ((((sigma**2)/(2*alpha))*(1-np.exp(-2*alpha*dt)))**0.5)*W[:,i+1]

    # print the results 
    print("The mean is:", np.mean(r[:,-1]))
    print("The varaiance is:", np.var(r[:,-1]))
    
    # count the negative valeus of r; print the total count
    for j in range(0, M):
        if any(x<0 for x in r[j,:]) == True:
            count += 1
    print("There are ", count, " negative trajectories out of a possible", M)
    
    # plot the trajectories
    t = np.linspace(0, S, N+1)
    for i in range(0,M):
        plt.plot(t, r[i,:])

In [50]:
interact(vasicek, r0 =(0,1,0.01))
plt.show()

interactive(children=(FloatSlider(value=0.03, description='r0', max=1.0, step=0.01), Output()), _dom_classes=(…

***Justtification of interval:*** $S=0, N = 1000$

**Accuary:*** a smaller time step will insure a more accurate simulation. Because we are working on a machine that can only work in finite time the smaller we choose $dt$ (i.e. more time steps $N$ larger). We choose N large enough so that we can get an accuarte answer, but also not too large so that it takes longer time to run

**Convergence:** Since $r(t)$ follows a vasicek model, $\alpha$ in our case can be taught of as the speed of mean-reversion; how quick the process reverts back to the mean. We want to chose a simulation time that gives the process enough time to revert back to it's mean. The half life of this proccess is $\frac{\log(2)}{\alpha}$, and for $\alpha = 1$ the half life is 0.69314... So we choose $S=10$ to ensure the process coverges to it's mean value of b

***Asymptotic Properties:*** Can clearly clearly see that when we move the slider for $r_0$, no matter what the value is the the values of r always converge to the mean of $b = 0.01$. Take $r_0 = 0.85$ as an example. All the trajectories initially go downwards, and start to converge around the mean of 0.01. Due to the mean being so close to 0; there is always going to be somne values that are **negative**. For our case the $Var[r(t)] = 0.01125$ which when will give a negative value for the first standard deviation away from the mean. 

Negtative interet rates are often viewed as a drawback of the VAsicek model, as negative rates are not favoured in an economic context. Central banks often use this unusual policy when there are strong signs of deflation. The ECB did this in 2014 to try and revive the Eurozone economy. A policy likes this charges commercial banks for holding excess reserves - encouraging them to lend more money.

### Question 2

#### 3D plot of bivariate gaussian pair $\left(r(T), \int_{0}^{T} r(s) ds\right)$

It can be shown that $Y(t) = \int_0^t r(s) ds$ has conditinal density:
$$
(Y(t) | r(s)) \sim \mathcal{N}\left(e^{-\alpha(t-s)}r_s + m_r(u,t), \sigma^2_Y2(u,t)\right)
$$
where
$$
m_Y(u, t) = \frac{1}{\alpha} (r_u + b) \left(1 - e^{-\alpha(t-u)}\right) + b(t - u),
$$
and
$$
\sigma^2_Y(u, t) = \frac{\sigma^2}{\alpha^2} \left( (t - u) + \frac{1}{2\alpha} \left(1 - e^{-2\alpha(t-u)}\right) + \frac{2}{\alpha} \left(e^{-\alpha(t-u)} - 1\right) \right).
$$

Moreover, the covariance of the increments of $r$ and $Y$ is given by
$$
\sigma_{r,Y} (u, t) = \frac{\sigma^2}{2\alpha} \left(1 + e^{-2\alpha(t-u)} - 2e^{-\alpha(t-u)}\right),
$$

with corresponding correltion
$$
\rho_{r,Y} (u, t) = \frac{\sigma_{r,Y}(u, t)}{\sigma_r(u, t)\sigma_Y(u, t)}
$$

Can find solutions to sample paths of the gaussian pair $\left(r(t), Y(t)\right)$ by the follwing:

\begin{equation}
\begin{bmatrix} r(t)\\ Y(t) \end{bmatrix} = 
\begin{bmatrix} e^{-\alpha(t-u)}r_s\\ Y_u \end{bmatrix} + \begin{bmatrix} m_t(u,t)\\ m_Y(u,t)\end{bmatrix} +
\begin{bmatrix} \sigma_r(u,t) & 0 \\ 0 & \sigma_Y(u,t)\end{bmatrix}A_{r,Y} Z
\end{equation}

where $Z = [Z_1, Z_2]^T$ is a pair of i.i.d RV where each $Z_i \sim\mathcal{N}\left(0,1\right)$

In [20]:
# Define parameters:
S = 10; N = 100; dt = S/N; alpha = 1; b = 0.03; sigma = 0.2; r0 = 0.04

# Define the functions needed for simulation
def mr(dt):
    return b*(1-np.exp(-alpha*dt))
              
def sigmar(dt):
    return np.sqrt((1-np.exp(-2*alpha*dt))*(sigma**2)/(2*alpha))
              
def mY(dt):
    return (1/alpha)*(r0+b)*(1-np.exp(-alpha*dt))+b*dt
              
def sigmaY(dt):
    return (sigma/alpha)*np.sqrt(dt+(0.5/alpha)*(1-np.exp(-2*alpha*dt))+(2/alpha)*(np.exp(-alpha*dt)-1))
              
def covrY(dt):
    return ((sigma**2)/(2*alpha))*(1+np.exp(-2*alpha*dt)-2*np.exp(-alpha*dt))
              
def rhorY(dt):
    return covrY(dt)/(sigmar(dt)*sigmaY(dt))


In [12]:
def interactive(r0 = 0.04):
    T = 0.5
    N1 = int(T/dt)
    Y0 = r0*T
    
    M = 500
    rmean = rvar = np.zeros(M)
    ymean = yvar = np.zeros(M)
    cor = np.zeros(M)
    
    Z = np.random.normal(0,1, (2,M))
    s=T*np.array([[1,rhorY(T)],[rhorY(T),1]])
    A=np.linalg.cholesky(s)   
    
    rY1=A.dot(Z)
    rY2=A.dot(Z)
    
    rY2[0,:]=np.exp(-alpha*T)*r0+mr(T)+sigmar(T)*rY1[0,:]
    rY2[1,:]=Y0+mY(T)+sigmaY(T)*rY1[1,:]
    
    mu_r = np.mean(rY2[0,:])
    mu_y = np.mean(rY2[1,:])
    var_r = np.var(rY2[0,:]); sd_r = np.sqrt(var_r)
    var_y = np.var(rY2[1,:]); sd_y = np.sqrt(var_y)
    corr = np.cov(rY2[0,:], rY2[1,:])
    
    x = np.linspace(-0.25, 0.25, 100)
    y = np.linspace(-0.1, 0.3, 100)
    X, Y = np.meshgrid(x, y)
    pos = np.dstack((X, Y))
    mu = np.array([mu_r, mu_y])
    cov = np.array([[var_r,corr[0,1]], [corr[1,0], var_y]])
    rv = multivariate_normal(mu, cov)
    Z = rv.pdf(pos)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    ax.plot_surface(X, Y, Z)
    ax.set_xlabel("$r(t)$")
    ax.set_ylabel("$\int_0^t r(t)$")
    ax.set_zlabel("Density")
    ax.set_title("Joint Distribution")
    plt.show()
    

In [21]:
interact(interactive, r0 = (-0.5,0.5, 0.01)) # negative values also given as vasicek can give negative values!
plt.show()

interactive(children=(FloatSlider(value=0.04, description='r0', max=0.5, min=-0.5, step=0.01), Output()), _dom…

#### NOTE: values for the x-axis and the y-axis were chosen to fit the curve in the center when plotting for $r_0 = 0.3$

## Question 3

#### Monte Carlo Valuation of Call Option on Zero Coupon Bond

Price of call option is given by: 
$$ P(r, t, S) = e^{A(t,S) - B(t,S)r}$$
where 
$$
B(t,S) = \frac{1}{\alpha} \left(1-e^{-\alpha(t-S)}\right)\\
A(t,s) =(B(t,S) - (S-t) \left(b - \frac{\sigma^2}{2\alpha^2}\right) - \frac{\sigma^2}{4\alpha} B(t,S)^2
$$

In [51]:
# intialise values
S = 10; N = 500; dt = S/N; alpha = 1; b = 0.04; sigma = 0.15; r0 = 0.03

# Define the functions needed for simulation
def mr(dt):
    return b*(1-np.exp(-alpha*dt))
              
def sigmar(dt):
    return np.sqrt((1-np.exp(-2*alpha*dt))*(sigma**2)/(2*alpha))
              
def mY(dt):
    return (1/alpha)*(r0+b)*(1-np.exp(-alpha*dt))+b*dt
              
def sigmaY(dt):
    return (sigma/alpha)*np.sqrt(dt+(0.5/alpha)*(1-np.exp(-2*alpha*dt))+(2/alpha)*(np.exp(-alpha*dt)-1))
              
def covrY(dt):
    return ((sigma**2)/(2*alpha))*(1+np.exp(-2*alpha*dt)-2*np.exp(-alpha*dt))
              
def rhorY(dt):
    return covrY(dt)/(sigmar(dt)*sigmaY(dt))


In [52]:
S = 10; N = 100; dt = int(S/N); alpha = 1; b = 0.04; sigma = 0.15; r0 = 0.03; M = 1000


r = np.zeros(M)
y = np.zeros(M)
A = np.zeros(M)
B = np.zeros(M)
P = np.zeros(M)

In [53]:
# Define function that will return the mean price of call option
def bond_option_price(r0, T, S, K):
    N = 100
    dt = T/N
    t = np.linspace(0, T, int(N))
    r[0] = r0
    y[0] = r[0] * T
    B[0] = 1/alpha * (1-np.exp(-alpha*S))
    A[0] = (B[0] - S) * (b - (sigma**2)/(S*alpha**2)) - ((sigma**2)/4*alpha) * B[0]**2
    
    # Monte carlo Evaluation 
    M = 1000
    final_price = np.zeros(M)
    for j in range(M):
        Z = np.random.normal(0,1, (2,N+1))
        for i in range(N-1):
            r[i+1] = np.exp(-alpha*dt)*r[i] + mr(dt) + sigmar(dt)*Z[0,i+1]
            y[i+1] = y[i] + mY(dt)*r[i] + (b/alpha)*(alpha*dt - 1 + np.exp(-alpha*dt)) + sigmaY(dt)*(rhorY(dt)*Z[0,i+1] + ((((1-rhorY(dt))**2)**0.5) * Z[1,i+1]))
            B[i+1] = (1/alpha)*(1-np.exp(-alpha*(S-t[i])))
            A[i+1] = (B[i]-(S-t[i])) * (b-((sigma**2)/(2*(alpha**2)))) - (sigma**2/4*alpha)*(B[i]**2)
            P[i+1] = np.exp(A[i] - B[i]*r[i])
        final_price[j] = P[N-1]
        
    # Compute the necessary Statistics    
    value = np.exp(-1*y) * np.maximum(final_price-K, 0)
    mean_price = np.mean(value)
    sd_price = np.std(value)
    CI_price = [mean_price - 1.96*(sd_price/np.sqrt(M)), mean_price + 1.96*(sd_price/np.sqrt(M))]
    CI_len  = CI_price[1] = CI_price[0]
    print("Mean value of Call Option is:", mean_price, 
          "\nStadard Deciation of Call Option is:", sd_price,
          "\n95% Confidence Interval for Call Option is:", CI_price, 
          "\nLength of Confidence Interval: ", CI_len
          )

In [56]:
bond_option_price(0.04,  0.5, 1, 1)

Mean value of Call Option is: 0.005747418327311906 
Stadard Deciation of Call Option is: 0.012912831258065229 
95% Confidence Interval for Call Option is: [0.004947072754100595, 0.004947072754100595] 
Length of Confidence Interval:  0.004947072754100595
